# execution

In [1]:
import pandas as pd
import numpy as np

In [2]:
L='APC FTI HPQ SANM SIG TEX TIF TJX'.upper().split(' ')

In [3]:
df=pd.read_csv('12.17-12.31.csv',header=0)

In [4]:
def execution(target_percentage,size,real_volume,real_price):
    i=0
    VWAP=0
    size_=size
    history=[]
    while size>0 and i<len(real_volume):
        goal=target_percentage[i]*size_
        if i==len(real_volume)-1:
            
            realized=min(size,real_volume[i])
        else:
            realized=min(goal,real_volume[i])
        realized=min(realized,size)
        VWAP+=realized*real_price[i]
        history.append(realized)
        
        i+=1
        size-=realized
    history=np.array(history)
    return history,size,VWAP/float(np.sum(history))
def benchmark(size,real_volume,real_price):
    history=np.array(real_volume)*size/float(sum(real_volume))
    VWAP=np.sum(history*np.array(real_price))
    return history,VWAP/float(np.sum(history))

In [5]:
def vol_pre(table, ticker):
    df=pd.read_csv(table,header=0)
    vol_pre=df[ticker]
    return vol_pre

In [8]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

def VWAP_calc(ticker,capital):
    df_t=df[df['SYM_ROOT']==ticker]
    D=sorted(list(set(df_t['DATE'])))
    St=[]
    S_t=[]
    for d in D[:-1]:
        i=-1
        s=[]
        T=[]
        df_d=df_t[df_t['DATE']==d]
        for j in range(len(df_d['TIME_M'])):
            
            if len(T)>0 and (int(np.array(df_d['TIME_M'])[j][-15:-13])-int(T[-1][-2:]))>1:
                    s.append(0)
                   
            if np.array(df_d['TIME_M'])[j][:-13] not in T:
                i+=1
                T.append(np.array(df_d['TIME_M'])[j][:-13])
                s.append(np.array(df_d['SIZE'])[j])
            else:
                s[i]+=np.array(df_d['SIZE'])[j]
        if '9:30' not in T:
            s=[0]+s
            
        if len(s)==30:
            St+=s
#             S_t.append(s)
    St=np.array(St,dtype=float)
#     S_t=np.array(S_t,dtype=float)
#     for i in range(len(S_t)):
#         S_t[i,:]=S_t[i,:]/float(np.sum(S_t[i,:]))
#     hist_mean=np.mean(S_t,axis=0)
#     T=[]
#     for i in range(int(len(St)/30)):
#         for j in range(30):
#             T.append(j)
#     T=np.array(T)
#     #print(T)
#     X=[]
#     for i in range(len(T)):
#         t=T[i]
#         X.append([t,t**2,t**3,t**4,t**5])
#     X=np.array(X)
#     regr = linear_model.LinearRegression()

#     # Train the model using the training sets
#     regr.fit(X,St)
#     pred_vol=regr.predict(X[:30,:])
    
#     pred_mean=pred_vol/np.sum(pred_vol)
    #print(vol_mean)
    df_d=df_t[df_t['DATE']==D[0]]
    T=[]
    s_start=[]
    p_start=[]
    i=-1
    for j in range(len(df_d['TIME_M'])):
        
        if len(T)>0 and (int(np.array(df_d['TIME_M'])[j][-15:-13])-int(T[-1][-2:]))>1:
            
                s_start.append(0)
                p_start.append(p_start[-1])
        if np.array(df_d['TIME_M'])[j][:-13] not in T:
            
            i+=1
            T.append(str(np.array(df_d['TIME_M'])[j][:-13]))
            p_start.append(np.array(df_d['PRICE'])[j])
            s_start.append(np.array(df_d['SIZE'])[j])
        else:
            s_start[i]+=np.array(df_d['SIZE'])[j]
    if '9:30' not in T:
        s_start=[0]+s_start
        p_start=[0]+p_start
    df_d=df_t[df_t['DATE']==D[-1]]
    T=[]
    s_end=[]
    p_end=[]
    i=-1
    for j in range(len(df_d['TIME_M'])):
        
        if len(T)>0 and (int(np.array(df_d['TIME_M'])[j][-15:-13])-int(T[-1][-2:]))>1:
           
                s_end.append(0)
                p_end.append(p_end[-1])
        if np.array(df_d['TIME_M'])[j][:-13] not in T:
            i+=1
            T.append(str(np.array(df_d['TIME_M'])[j][:-13]))
            p_end.append(np.array(df_d['PRICE'])[j])
            s_end.append(np.array(df_d['SIZE'])[j])
        else:
            s_end[i]+=np.array(df_d['SIZE'])[j]
    if '9:30' not in T:
        s_end=[0]+s_end
        p_end=[0]+p_end
    #print(T)
    vol_pre_12_17=vol_pre('12.17_vol_pre_pct.csv',ticker)
    vol_pre_12_31=vol_pre('12.31_vol_pre_pct.csv',ticker)
    avg_vol=np.ones(30)/30
    Dict_=np.load('data.npy').item()
    p_d=np.array(Dict_[ticker.lower()]['Open'])[11]
    p_s=np.array(Dict_[ticker.lower()]['Open'])[23]
    cap=capital
    size=int(cap/p_d)
    hb,r,pb=execution(vol_pre_12_17,size,s_start,p_start)
    size=int(cap/p_d)
    hs,r,ps=execution(vol_pre_12_31,size,s_end,p_end)
    size=int(cap/p_d)
    hb,r,pb2=execution(avg_vol,size,s_start,p_start)
    size=int(cap/p_d)
    hs,r,ps2=execution(avg_vol,size,s_end,p_end)
    h,bb=benchmark(size,s_start,p_start)
    h,bs=benchmark(size,s_end,p_end)
    print('2018.12.17 (Buy)Decision Price:'+str(p_d)+' Executed VWAP(volume prediction):'+str(round(pb,3))+' Executed VWAP(average volume):'+str(round(pb2,3))+' Benchmark VWAP:'+str(round(bb,3)))
    print('2018.12.31 (Sell)Decision Price:'+str(p_s)+' Executed VWAP(volume prediction):'+str(round(ps,3))+' Executed VWAP(average volume):'+str(round(ps2,3))+' Benchmark VWAP:'+str(round(bs,3)))
    print('IS1:'+str(((pb-p_d)*size+(p_s-ps)*size)/float(capital))+' IS2:'+str(((pb2-p_d)*size+(p_s-ps2)*size)/float(capital)))
    return St,(size*ps-size*pb)/(size*pb),hb,hs,pb,ps,bb,bs,((bb-p_d)*size+(p_s-bs)*size)/float(capital),((pb-p_d)*size+(p_s-ps)*size)/float(capital),((pb2-p_d)*size+(p_s-ps2)*size)/float(capital)

In [9]:
shortfall=[]
shortfall2=[]
benchmark_sf=[]
for i in set(L):
  
    print(i)
    record=VWAP_calc(i,L.count(i)*100000)
    IS=record[-1]
    IS2=record[-2]
    bm=record[-3]
    shortfall.append(IS)
    shortfall2.append(IS2)
    benchmark_sf.append(bm)

SIG
2018.12.17 (Buy)Decision Price:56.926 Executed VWAP(volume prediction):31.769 Executed VWAP(average volume):31.749 Benchmark VWAP:31.774
2018.12.31 (Sell)Decision Price:54.8916 Executed VWAP(volume prediction):31.269 Executed VWAP(average volume):31.278 Benchmark VWAP:31.217
IS1:-0.02694260104419838 IS2:-0.02744958013333344
APC
2018.12.17 (Buy)Decision Price:67.2917 Executed VWAP(volume prediction):50.401 Executed VWAP(average volume):50.413 Benchmark VWAP:50.43
2018.12.31 (Sell)Decision Price:52.5223 Executed VWAP(volume prediction):43.754 Executed VWAP(average volume):43.784 Benchmark VWAP:43.764
IS1:-0.12069194103616473 IS2:-0.1209676814000001
HPQ
2018.12.17 (Buy)Decision Price:23.0682 Executed VWAP(volume prediction):21.929 Executed VWAP(average volume):21.93 Benchmark VWAP:21.944
2018.12.31 (Sell)Decision Price:24.0737 Executed VWAP(volume prediction):20.524 Executed VWAP(average volume):20.528 Benchmark VWAP:20.521
IS1:0.10447652082656808 IS2:0.10433989426666662
TIF
2018.12.1

In [11]:
print('Period Theoretical Return:'+str(0.0836137916348646))
print('Volume pre Shortfall:'+str(np.mean(shortfall2)))
print('Average vol Shortfall:'+str(np.mean(shortfall)))
print('Benchmark Shortfall:'+str(np.mean(benchmark_sf)))

Period Theoretical Return:0.0836137916348646
Volume pre Shortfall:-0.017791131551633848
Average vol Shortfall:-0.017821982136915656
Benchmark Shortfall:-0.016830971165967712
